<a href="https://colab.research.google.com/github/cabamarcos/SuperMask/blob/main/MNIST_Resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau
from tensorflow.keras.datasets import mnist

import os


In [ ]:

(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Define the labels of the dataset
labels = ['0', '1', '2', '3', '4',
          '5', '6', '7', '8', '9']

# Let's view more images in a grid format
# Define the dimensions of the plot grid
W_grid = 10
L_grid = 10

# fig, axes = plt.subplots(L_grid, W_grid)
# subplot return the figure object and axes object
# we can use the axes object to plot specific figures at various locations

fig, axes = plt.subplots(L_grid, W_grid, figsize = (17,17))

axes = axes.ravel() # flaten the 15 x 15 matrix into 225 array

n_train = len(X_train) # get the length of the train dataset

# Select a random number from 0 to n_train
for i in np.arange(0, W_grid * L_grid): # create evenly spaces variables

    # Select a random number
    index = np.random.randint(0, n_train)
    # read and display an image with the selected index
    axes[i].imshow(X_train[index,1:])
    label_index = int(y_train[index])
    axes[i].set_title(labels[label_index], fontsize = 8)
    axes[i].axis('off')

plt.subplots_adjust(hspace=0.4)

In [ ]:
classes_name =['0', '1', '2', '3', '4','5', '6', '7', '8', '9']

classes, counts = np.unique(y_train, return_counts=True)
plt.barh(classes_name, counts)
plt.title('Class distribution in training & test')

classes, counts = np.unique(y_test, return_counts=True)
plt.barh(classes_name, counts)


In [ ]:
# Applying Data Augmentation

datagen = ImageDataGenerator(rotation_range=10,  zoom_range = 0.1,  width_shift_range=0.1,  height_shift_range=0.1)
# datagen.fit(X)

In [ ]:
# Learning rate annealer

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
                                patience=3,
                                verbose=1,
                                factor=0.2,
                                min_lr=1e-6)

In [ ]:
model = keras.models.Sequential([
keras.layers.Conv2D(32, kernel_size = 3, activation = 'relu', padding = 'same', input_shape = [28, 28, 1]),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(64, kernel_size = 3, activation = 'relu', padding = 'same'),
keras.layers.MaxPool2D(),
keras.layers.Conv2D(32, kernel_size = 3, activation = 'relu', padding = 'same'),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(64, kernel_size = 3, activation = 'relu', padding = 'same'),
keras.layers.MaxPool2D(),
keras.layers.Flatten(),
keras.layers.Dropout(0.25),
keras.layers.Dense(256, activation = 'relu'),
keras.layers.Dropout(0.5),
keras.layers.Dense(10, activation = 'softmax')])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'nadam', metrics = ['accuracy'])

In [ ]:
X_train = X_train.reshape((60000, 28, 28, 1))


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.08)
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = 64),
                              epochs = 25, validation_data = (X_valid, y_valid), callbacks = [reduce_lr])
print("Maximum Train Accuracy : {} Maximum Validation Accuracy : {}".format(max(history.history['accuracy']), max(history.history['val_accuracy'])))


In [ ]:
X_test = X_test.reshape((10000, 28, 28, 1))


In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)
